<a href="https://colab.research.google.com/github/shuntarotobari/qtest/blob/main/Growthtable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd

# Load data
pwt = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt100.dta')

# Filter and select relevant variables
countries = ['AUS', 'AUT', 'BEL', 'CAN', 'DNK', 'FIN', 'FRA', 'DEU', 'GRC', 'ISL', 'IRL', 'ITA', 'JPN', 'NLD', 'NZL', 'NOR', 'PRT', 'ESP', 'SWE', 'CHE', 'GBR', 'USA']
variables = ['countrycode', 'year', 'rgdpna', 'rkna', 'emp', 'avh', 'labsh']
data = pwt.loc[(pwt['countrycode'].isin(countries)) & (pwt['year'] >= 1960) & (pwt['year'] <= 2000), variables]

# Calculate additional variables
data = data.sort_values(['countrycode', 'year'])
data['alpha'] = 1 - data['labsh']
data['y_n'] = data['rgdpna'] / data['emp']
data['hours'] = data['emp'] * data['avh']
data['tfp_term'] = data.groupby('countrycode')['rgdpna'].pct_change().shift(-1) / (1 - data['alpha'])
data['cap_term'] = (data['rkna'] / data['rgdpna']).groupby(data['countrycode']).pct_change().shift(-1) * (data['alpha'] / (1 - data['alpha']))
data = data.dropna()

# Calculate growth rates
data['growth_rate'] = data.groupby('countrycode')['y_n'].pct_change()
data['tfp_growth'] = data['tfp_term']
data['capital_deepening'] = data['cap_term']

# Calculate tfp_share and capital_share directly
data['tfp_share'] = data['tfp_growth'] / data['growth_rate']
data['capital_share'] = 1 - data['tfp_share']

# Calculate averages
averages = data.groupby('year')[['growth_rate', 'tfp_growth', 'capital_deepening', 'tfp_share', 'capital_share']].mean().reset_index()
averages = averages.rename(columns={'countrycode': 'Country'})

# Print the table
result = pd.DataFrame({
    'Country': data['countrycode'].unique(),
    'Growth rate': data.groupby('countrycode')['growth_rate'].mean(),
    'TFP Growth': data.groupby('countrycode')['tfp_growth'].mean(),
    'Capital Deepening': data.groupby('countrycode')['capital_deepening'].mean(),
    'TFP share': data.groupby('countrycode')['tfp_share'].mean(),
    'Capital share': data.groupby('countrycode')['capital_share'].mean()
})

averages['Country'] = averages.index
print(pd.concat([result, averages], sort=False))

    Country  Growth rate  TFP Growth  Capital Deepening  TFP share  \
AUS     AUS     0.018057    0.056813           0.004195 -28.855717   
AUT     AUT     0.030875    0.052312           0.006048  -2.761757   
BEL     BEL     0.027483    0.049347           0.001610   3.459938   
CAN     CAN     0.015189    0.050220           0.003346   3.035695   
CHE     CHE     0.013923    0.034767           0.005893  -4.024617   
..      ...          ...         ...                ...        ...   
35       35          NaN         NaN                NaN        NaN   
36       36          NaN         NaN                NaN        NaN   
37       37          NaN         NaN                NaN        NaN   
38       38          NaN         NaN                NaN        NaN   
39       39          NaN         NaN                NaN        NaN   

     Capital share    year  growth_rate  tfp_growth  capital_deepening  \
AUS      29.855717     NaN          NaN         NaN                NaN   
AUT       3